# (적용 X) BERT 코드

In [ ]:
import pandas as pd
import numpy as np
import os
import pickle
from glob import glob
from itertools import permutations
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, log_loss

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour, plot_optimization_history
from optuna.visualization import plot_parallel_coordinate, plot_slice, plot_param_importances

from bert_sklearn import BertClassifier
from sklearn.neural_network import MLPClassifier
from catboost import Pool, CatBoostClassifier
import xgboost as xgb

In [ ]:
train = pd.read_csv("../data/train.csv", index_col="id")
test = pd.read_csv("../data/test.csv", index_col="id")
submission = pd.read_csv("../data/sample_submission.csv")

In [ ]:
def create_dir(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
        print("Created Directory :", dir)
    else:
        print("Directory already existed :", dir)
create_dir("../pickle")
create_dir("../model")
create_dir("../submission")

In [ ]:
train_x = train['text']
train_y = train['target']
test_x = test['text']

In [ ]:
rows_train = train.shape[0] # 주어진 train data의 row 수
rows_test = test.shape[0] # 주어진 test data의 row 수
num_classes = len(train_y.unique())
num_trial = 100 # 파라미터 튜닝을 몇 번 진행하는지의 수
splits_hp = 3 # 파라미터 튜닝을 진행할 때의 kfold 수
splits_tr = 15 # 모델 트레이닝을 진행할 때의 kfold 수
basic_seed = 42 # default seed
num_seed_tr = 5 # 트레이닝 seed 개수
sel_seed = 3 # 선택할 seed 개수

In [ ]:
pred_dict = {}
pred_test_dict = {}

In [ ]:
lucky_seeds = np.random.randint(0, 1000, num_seed_tr)

for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=splits_tr, random_state=seed, shuffle=True)
    cv = np.zeros((rows_train, num_classes))
    pred_test = np.zeros((rows_test, num_classes))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx].values.ravel(), train_y.iloc[val_idx].values.ravel()
        
#         print(f'fold {n+1} start')
        
        BERTModel = BertClassifier(bert_model="bert-base-cased", random_state=basic_seed,
                                   epochs=5, validation_fraction=0, train_batch_size=8, eval_batch_size=2)
        BERTModel.fit(x_train, y_train)
        
        cv[val_idx, :] = BERTModel.predict_proba(x_val)
        pred_test += BERTModel.predict_proba(test_x) / splits_tr
        
        print(f'fold {n+1}', 'log_loss :', log_loss(y_val, cv[val_idx]))
        print(f'fold {n+1}', 'accuracy_score :', accuracy_score(y_val, np.argmax(cv[val_idx], axis=1)))
        
    pred_dict['bert'+str(seed)] = cv
    pred_test_dict['bert'+str(seed)] = pred_test
    print(f'seed {seed}', 'log_loss :', log_loss(train_y, cv))
    print(f'seed {seed}', 'accuracy_score :', accuracy_score(train_y, np.argmax(cv, axis=1)))

In [ ]:
lucky_seeds = np.random.randint(0, 1000, num_seed_tr)

for i, seed in enumerate(lucky_seeds):

    kfold = StratifiedKFold(n_splits=splits_tr, random_state=seed, shuffle=True)
    cv = np.zeros((rows_train, num_classes))
    pred_test = np.zeros((rows_test, num_classes))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):
        
        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx].values.ravel(), train_y.iloc[val_idx].values.ravel()
        
        vectorizer = TfidfVectorizer(ngram_range=(1, 2))
        vectorizer.fit(x_train)
        x_train = vectorizer.transform(x_train)
        x_val = vectorizer.transform(x_val)
        x_test = vectorizer.transform(test_x)
        
#         print(f'fold {n+1} start')
        
        MLPModel = MLPClassifier(max_iter=3, hidden_layer_sizes=250, random_state=basic_seed, verbose=False)
        MLPModel.fit(x_train, y_train)
        
        cv[val_idx, :] = MLPModel.predict_proba(x_val)
        
        pred_test += MLPModel.predict_proba(x_test) / splits_tr
        
        print(f'fold {n+1}', 'log_loss :', log_loss(y_val, cv[val_idx]))
        print(f'fold {n+1}', 'accuracy_score :', accuracy_score(y_val, np.argmax(cv[val_idx], axis=1)))
        
    pred_dict['mlp'+str(seed)] = cv
    pred_test_dict['mlp'+str(seed)] = pred_test
    print(f'seed {seed}', 'log_loss :', log_loss(train_y, cv))
    print(f'seed {seed}', 'accuracy_score :', accuracy_score(train_y, np.argmax(cv, axis=1)))

In [ ]:
def sort_dict(model, pred_dict, pred_test_dict):
    pred_dict_local = {}
    for key, value in pred_dict.items():
        if model in key:
            pred_dict_local[key]=value

    pred_test_dict_local = {}
    for key, value in pred_test_dict.items():
        if model in key:
            pred_test_dict_local[key]=value

    pred_dict_new_local = dict(sorted(
        pred_dict_local.items(), 
        key=lambda x:accuracy_score((train_y), np.argmax(list(x[1]), axis=1)), reverse=False)[:5])
    pred_test_dict_new_local = {}
    for key, value in pred_dict_new_local.items():
        pred_test_dict_new_local[key]=pred_test_dict_local[key]
        
    return pred_dict_new_local, pred_test_dict_new_local

In [ ]:
def save_dict(model, pred_dict, pred_test_dict):
    with open('../pickle/pred_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_dict, fw)
    with open('../pickle/pred_test_dict_'+model+'.pickle', 'wb') as fw:
        pickle.dump(pred_test_dict, fw)

In [ ]:
pred_dict_bert, pred_test_dict_bert = sort_dict('bert', pred_dict, pred_test_dict)
save_dict('bert_epoch5_cv15', pred_dict_bert, pred_test_dict_bert)

In [ ]:
pred_dict_mlp, pred_test_dict_mlp = sort_dict('mlp', pred_dict, pred_test_dict)
save_dict('mlp_epoch3_layer250_cv15', pred_dict_mlp, pred_test_dict_mlp)

In [ ]:
pred = np.zeros((rows_train, num_classes))
for _, value in pred_dict_mlp.items():
    pred += value
pred /= len(pred_dict_mlp)

In [ ]:
pred2 = np.zeros((rows_train, num_classes))
for _, value in pred_dict_bert.items():
    pred2 += value
pred2 /= len(pred_dict_bert)

In [ ]:
pred_test = np.zeros((rows_test, num_classes))
for _, value in pred_test_dict_mlp.items():
    pred_test += value
pred_test /= len(pred_test_dict_mlp)

In [ ]:
pred_test2 = np.zeros((rows_test, num_classes))
for _, value in pred_test_dict_bert.items():
    pred_test2 += value
pred_test2 /= len(pred_test_dict_bert)

In [ ]:
print(f'accuracy_score: {accuracy_score(train_y, np.argmax(pred*0.86+pred2*0.14, axis=1)):.6f}')

In [ ]:
pred_test = np.argmax(pred_test*0.86+pred_test2*0.14, axis=1)

In [ ]:
submission["target"] = pred_test

In [ ]:
submission['target'].value_counts()

In [ ]:
submission_name = '20220410'
submission_number = '1'
submission.to_csv(f'../submission/{submission_name}-{submission_number}.csv', index = False)

In [ ]:
submission